# Preparing A New Wiki Dump 

**Performing these steps can be tricky and there are better option [see README](../../README.md#hosting)**

* **Make sure you have alreay followed the first two steps of the setup explained in [README.md](../../README.md#hosting)**
* ** You can find most of the intermediate files in our website reporitory (the permament address can be found [here](../../README.md#api)) **
* Note that all of the paths are hard-coded and you need to modify them, consistently, and carefully!



But if you want to start frome scratch and import a new Wikipedia version, you should start from here 
to prepapre the graph structure. This is enough for calculating *concept embedding* and *semantic relatedness*. 
However, for the text databases, necessary for Word-sense disambiguation, you should proceed [here](../text/prepare_annonated_indexed_wiki.ipynb). 


## Preprocessing


The first step is to download the wikipedia database dumps and import them to mysql. We do a preprocessing on the sql dumps for mainly three reasons:

* The tables are huge, containing many column and rows we do not use. Removing the unnessary information, that includes unused columns (such as time stamps, viewed count of the pages or categories) and all the information about talk pages, media files or user draft pages, can dramatically decreas the size of the tables.

* Forming **synonym Rings**. We extend the concept of synonym ring to Wikipedia (similar to what is called synset in Wordnet). In Wikipedia, redirection stands for equivallency, for example Car --> Automobile. But it's not always this easy and you can find all sorts of weired redirection, like:

![](../../resrc/sr.jpg)

   We iterate through redirectins and remove cycles, dangling redirections and also all the chains. This process forms clusters of redirections around main pages. Then we go through all other tables (pagelinks and  category links) and replace any redirected page by its main article, the result would be much more neated, and makes the rest of the process faster.


* We remove garbage, links to non existing pages, self links, mismatching namespaces, and many other incosistencies that you can find the details in the source code).

* We apply some strategic changes, like instead of source id --> destination title format of the pagelinks, we use source id --> dest id, which is faster and preferrabel for out case. 

To complete this step, download and run the parser (written in Java) that prunes these files. You can run the following cells, but due to a known bug with ipython, you can't see bash progress messages untill the job is finished. So a better option would be simply running the scripts from bash and skipping the remaining of this section. In this case, running each cell create a script in the [preparation_scripts/] directory with the name indicated as the argument of `writefile` at the begining of the cell. If you want to run the cell directly, ** comment the first line and uncomment the second line**.

## Step 0. Installing MySQL (we use MariaDB)
Install and run the server, you may want to increase the variables (you can check our .my.cnf)

**All of the paths are hard-coded and arbitrary (and relative to the current directory), you need to change them consistently.**

## Step 1. Downloading

Download the following files and decompress them (we assume the path to be `../../../data/enwiki20160305/original`):

https://dumps.wikimedia.org/enwiki/20160305/enwiki-latest-page.sql.gz

https://dumps.wikimedia.org/enwiki/20160305/enwiki-latest-pagelinks.sql.gz 

https://dumps.wikimedia.org/enwiki/20160305/enwiki-latest-redirect.sql.gz

https://dumps.wikimedia.org/enwiki/20160305/enwiki-latest-category.sql.gz

https://dumps.wikimedia.org/enwiki/20160305/enwiki-latest-categorylinks.sql.gz

### Or
**Use the the prepared following script that download wikipedia dumps and decompress them**


`bash download.sh ../../../data/enwiki20160305/original`

`bash decompress.sh ../../../data/enwiki20160305/original`


## Step 2. Parsing Database dumps

The following java file  does the preprosseing (parsing) wikipedia dumps and creates the processed tables (ending in `main.sql`) and several log files of the errors

*Note*: you might need to recompile (`javac ProcessSQLDumps.java`) 

run by

`java ProcessSQLDumps ../../../data/enwiki20160305/original`


`mkdir ../../../data/enwiki20160305/edited`

`mv ../../../data/enwiki20160305/original/*.tsv ../../../data/enwiki20160305/edited/`

### Step 3. Preparing mysql
Running the folling cell will set some variable in mysql for maximum performance (if you have enoguh physical memory). Replace \$1 and \$2 with the actuall user and password of the user, or run the script as:

`bash setupmysql.sh <user> <pass>`


### Step 4. Actuall Importing

```mysql -u <user> -p<pass> -e 'CREATE SCHEMA `enwiki20160305` DEFAULT CHARACTER SET binary;'```

`./importall  ../../../data/enwiki20160305/edited last <user> <pass>`

This might take several hours 

### If you want to setup the Apache Solr server, continue [here](../text/prepare_annonated_indexed_wiki.ipynb)
